In [1]:
!pip install pyspark==3.1.2
!pip install delta-spark

In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '\
--packages io.delta:delta-core_2.12:1.0.0 \
--conf "spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension" \
--conf "spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog" \
pyspark-shell'

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr
from delta.tables import DeltaTable
import shutil

# Clear any previous runs
try:
    shutil.rmtree("/tmp/delta-table")
except:
    pass

# Enable SQL commands and Update/Delete/Merge for the current spark session.
# we need to set the following configs
spark = SparkSession.builder \
    .appName("quickstart1") \
    .getOrCreate()

# Create a table
print("############# Creating a table ###############")
data = spark.range(0, 5)
data.write.format("delta").save("/tmp/delta-table")

# Read the table
print("############ Reading the table ###############")
df = spark.read.format("delta").load("/tmp/delta-table")
df.show()

# Upsert (merge) new data
print("########### Upsert new data #############")
newData = spark.range(0, 20)

deltaTable = DeltaTable.forPath(spark, "/tmp/delta-table")

deltaTable.alias("oldData")\
    .merge(
    newData.alias("newData"),
    "oldData.id = newData.id")\
    .whenMatchedUpdate(set={"id": col("newData.id")})\
    .whenNotMatchedInsert(values={"id": col("newData.id")})\
    .execute()

deltaTable.toDF().show()

# Update table data
print("########## Overwrite the table ###########")
data = spark.range(5, 10)
data.write.format("delta").mode("overwrite").save("/tmp/delta-table")
deltaTable.toDF().show()

deltaTable = DeltaTable.forPath(spark, "/tmp/delta-table")

# Update every even value by adding 100 to it
print("########### Update to the table(add 100 to every even value) ##############")
deltaTable.update(
    condition=expr("id % 2 == 0"),
    set={"id": expr("id + 100")})

deltaTable.toDF().show()

# Delete every even value
print("######### Delete every even value ##############")
deltaTable.delete(condition=expr("id % 2 == 0"))
deltaTable.toDF().show()

# Read old version of data using time travel
print("######## Read old data using time travel ############")
df = spark.read.format("delta").option("versionAsOf", 0).load("/tmp/delta-table")
df.show()

# cleanup
shutil.rmtree("/tmp/delta-table")

############# Creating a table ###############
############ Reading the table ###############
+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+

########### Upsert new data #############
+---+
| id|
+---+
|  8|
|  1|
| 13|
|  5|
| 15|
| 17|
| 16|
|  0|
|  3|
| 12|
|  2|
| 11|
| 18|
|  4|
|  9|
|  7|
|  6|
| 14|
| 10|
| 19|
+---+

########## Overwrite the table ###########
+---+
| id|
+---+
|  5|
|  6|
|  7|
|  8|
|  9|
+---+

########### Update to the table(add 100 to every even value) ##############
+---+
| id|
+---+
|  5|
|106|
|  7|
|108|
|  9|
+---+

######### Delete every even value ##############
+---+
| id|
+---+
|  5|
|  7|
|  9|
+---+

######## Read old data using time travel ############
+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



In [7]:
from pyspark.sql import SparkSession

tableName = "tbltestpython"

# Enable SQL/DML commands and Metastore tables for the current spark session.
# We need to set the following configs

spark = SparkSession.builder \
    .appName("quickstart_sql1") \
    .master("local[*]") \
    .getOrCreate()

# Clear any previous runs
spark.sql("DROP TABLE IF EXISTS " + tableName)
spark.sql("DROP TABLE IF EXISTS newData")

try:
    # Create a table
    print("############# Creating a table ###############")
    spark.sql("CREATE TABLE %s(id LONG) USING delta" % tableName)
    spark.sql("INSERT INTO %s VALUES 0, 1, 2, 3, 4" % tableName)

    # Read the table
    print("############ Reading the table ###############")
    spark.sql("SELECT * FROM %s" % tableName).show()

    # Upsert (merge) new data
    print("########### Upsert new data #############")
    spark.sql("CREATE TABLE newData(id LONG) USING parquet")
    spark.sql("INSERT INTO newData VALUES 3, 4, 5, 6")

    spark.sql('''MERGE INTO {0} USING newData
            ON {0}.id = newData.id
            WHEN MATCHED THEN
              UPDATE SET {0}.id = newData.id
            WHEN NOT MATCHED THEN INSERT *
        '''.format(tableName))

    spark.sql("SELECT * FROM %s" % tableName).show()

    # Update table data
    print("########## Overwrite the table ###########")
    spark.sql("INSERT OVERWRITE %s select * FROM (VALUES 5, 6, 7, 8, 9) x (id)" % tableName)
    spark.sql("SELECT * FROM %s" % tableName).show()

    # Update every even value by adding 100 to it
    print("########### Update to the table(add 100 to every even value) ##############")
    spark.sql("UPDATE {0} SET id = (id + 100) WHERE (id % 2 == 0)".format(tableName))
    spark.sql("SELECT * FROM %s" % tableName).show()

    # Delete every even value
    print("######### Delete every even value ##############")
    spark.sql("DELETE FROM {0} WHERE (id % 2 == 0)".format(tableName))
    spark.sql("SELECT * FROM %s" % tableName).show()

    # Read old version of data using time travel
    print("######## Read old data using time travel ############")
    df = spark.read.format("delta").option("versionAsOf", 0).table(tableName)
    df.show()

finally:
    # cleanup
    spark.sql("DROP TABLE " + tableName)
    spark.sql("DROP TABLE IF EXISTS newData")
    spark.stop()

############# Creating a table ###############
############ Reading the table ###############
+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+

########### Upsert new data #############
+---+
| id|
+---+
|  3|
|  2|
|  4|
|  0|
|  1|
|  5|
|  6|
+---+

########## Overwrite the table ###########
+---+
| id|
+---+
|  5|
|  6|
|  7|
|  8|
|  9|
+---+

########### Update to the table(add 100 to every even value) ##############
+---+
| id|
+---+
|  5|
|106|
|  7|
|108|
|  9|
+---+

######### Delete every even value ##############
+---+
| id|
+---+
|  5|
|  7|
|  9|
+---+

######## Read old data using time travel ############
+---+
| id|
+---+
+---+



In [9]:
from pyspark.sql import SparkSession
import shutil

table_dir = "/tmp/delta-table"
# Clear any previous runs
try:
    shutil.rmtree(table_dir)
except:
    pass

# Enable SQL/DML commands and Metastore tables for the current spark session.
# We need to set the following configs

spark = SparkSession.builder \
    .appName("quickstart_sql_on_paths") \
    .master("local[*]") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Clear any previous runs
spark.sql("DROP TABLE IF EXISTS newData")

try:
    # Create a table
    print("############# Creating a table ###############")
    spark.sql("CREATE TABLE delta.`%s`(id LONG) USING delta" % table_dir)
    spark.sql("INSERT INTO delta.`%s` VALUES 0, 1, 2, 3, 4" % table_dir)

    # Read the table
    print("############ Reading the table ###############")
    spark.sql("SELECT * FROM delta.`%s`" % table_dir).show()

    # Upsert (merge) new data
    print("########### Upsert new data #############")
    spark.sql("CREATE TABLE newData(id LONG) USING parquet")
    spark.sql("INSERT INTO newData VALUES 3, 4, 5, 6")

    spark.sql('''MERGE INTO delta.`{0}` AS data USING newData
            ON data.id = newData.id
            WHEN MATCHED THEN
              UPDATE SET data.id = newData.id
            WHEN NOT MATCHED THEN INSERT *
        '''.format(table_dir))

    spark.sql("SELECT * FROM delta.`%s`" % table_dir).show()

    # Update table data
    print("########## Overwrite the table ###########")
    spark.sql("INSERT OVERWRITE delta.`%s` select * FROM (VALUES 5, 6, 7, 8, 9) x (id)" % table_dir)
    spark.sql("SELECT * FROM delta.`%s`" % table_dir).show()

    # Update every even value by adding 100 to it
    print("########### Update to the table(add 100 to every even value) ##############")
    spark.sql("UPDATE delta.`{0}` SET id = (id + 100) WHERE (id % 2 == 0)".format(table_dir))
    spark.sql("SELECT * FROM delta.`%s`" % table_dir).show()

    # Delete every even value
    print("######### Delete every even value ##############")
    spark.sql("DELETE FROM delta.`{0}` WHERE (id % 2 == 0)".format(table_dir))
    spark.sql("SELECT * FROM delta.`%s`" % table_dir).show()
    
finally:
    # cleanup
    spark.sql("DROP TABLE IF EXISTS newData")
    spark.stop()

############# Creating a table ###############
############ Reading the table ###############
+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+

########### Upsert new data #############
+---+
| id|
+---+
|  0|
|  3|
|  2|
|  1|
|  5|
|  4|
|  6|
+---+

########## Overwrite the table ###########
+---+
| id|
+---+
|  5|
|  6|
|  7|
|  8|
|  9|
+---+

########### Update to the table(add 100 to every even value) ##############
+---+
| id|
+---+
|  5|
|106|
|  7|
|108|
|  9|
+---+

######### Delete every even value ##############
+---+
| id|
+---+
|  5|
|  7|
|  9|
+---+



In [11]:
from pyspark.sql import SparkSession
from delta.tables import DeltaTable
import shutil

spark = SparkSession.builder \
    .appName("utilities") \
    .master("local[*]") \
    .getOrCreate()

# Clear previous run's delta-tables
try:
    shutil.rmtree("/tmp/delta-table")
except:
    pass

# Create a table
print("########### Create a Parquet table ##############")
data = spark.range(0, 5)
data.write.format("parquet").save("/tmp/delta-table")

# Convert to delta
print("########### Convert to Delta ###########")
DeltaTable.convertToDelta(spark, "parquet.`/tmp/delta-table`")

# Read the table
df = spark.read.format("delta").load("/tmp/delta-table")
df.show()

deltaTable = DeltaTable.forPath(spark, "/tmp/delta-table")
print("######## Vacuum the table ########")
deltaTable.vacuum()

print("######## Describe history for the table ######")
deltaTable.history().show()

# Generate manifest
print("######## Generating manifest ######")
deltaTable.generate("SYMLINK_FORMAT_MANIFEST")

# SQL Vacuum
print("####### SQL Vacuum #######")
spark.sql("VACUUM '%s' RETAIN 169 HOURS" % "/tmp/delta-table").collect()

# SQL describe history
print("####### SQL Describe History ########")
print(spark.sql("DESCRIBE HISTORY delta.`%s`" % ("/tmp/delta-table")).collect())

# cleanup
shutil.rmtree("/tmp/delta-table")

########### Create a Parquet table ##############
########### Convert to Delta ###########
+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+

######## Vacuum the table ########
######## Describe history for the table ######
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|version|           timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|      0|2021-10-18 08:48:...|  null|    null|  CONVERT|{numFiles -> 1, p...|null|    null|     null|         -1|  Serializable|        false|{numConvertedFile...|        null|
+-------+--------------------+------+--------+---------+--------